In [ ]:
import time
import pygame
import math3d as m3d
from math import pi
import urx
import rtde_control
import rtde_io
import math
import rtde_receive

In [ ]:
class Cmd(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.axis0 = 0
        self.axis1 = 0
        self.axis2 = 0 
        self.axis3 = 0
        self.axis4 = 0
        self.axis5 = 0 
        self.btn0 = 0
        self.btn1 = 0
        self.btn2 = 0
        self.btn3 = 0
        self.btn4 = 0
        self.btn5 = 0
        self.btn6 = 0
        self.btn7 = 0
        self.btn8 = 0
        self.btn9 = 0
        self.hat0 = 0

class Service(object):
    def __init__(self, robot, robo_io,rtde_r, linear_velocity=0.1, rotational_velocity=0.1, acceleration=0.1):
        self.joystick = None
        self.robot = robot
        self.robo_io = robo_io
        self.robo_r = rtde_r
        self.linear_velocity = linear_velocity
        self.rotational_velocity = rotational_velocity
        self.acceleration = acceleration
        self.init_pose = [0,-math.pi/2,math.pi/2,0,math.pi/2,math.pi]
        self.cmd = Cmd()

    def init_joystick(self):
        pygame.init()
        self.joystick = pygame.joystick.Joystick(0)
        self.joystick.init()
    
    def move_robo(self,pose_wrt_tool):
        robo_actual_pose = self.robo_r.getActualTCPPose()
        rel_robo_actual_pose = [0,0,0,robo_actual_pose[3],robo_actual_pose[4],robo_actual_pose[5]]
        speed_vect = self.robot.poseTrans(rel_robo_actual_pose,pose_wrt_tool)
        t_speed_vect = [speed_vect[0],speed_vect[1],speed_vect[2],0,0,0]
        self.robot.speedL(t_speed_vect,1.0,1.0)
        print(t_speed_vect)
    
    def rotate_robo(self,pose_wrt_tool):
        robo_actual_pose = self.robo_r.getActualTCPPose()
        rel_robo_actual_pose = [robo_actual_pose[0],robo_actual_pose[1],robo_actual_pose[2],0,0,0]
        speed_vect = self.robot.poseTrans(rel_robo_actual_pose,pose_wrt_tool)
        t_speed_vect = [0,0,0,speed_vect[3],speed_vect[4],speed_vect[5]]
        self.robot.speedL(t_speed_vect,0.5,0.5)
        #print(t_speed_vect)

    
    def test(self,wet):
        print(wet)
    
    def loop(self):
        air = False
        while True:
            self.cmd.reset()
            pygame.event.pump()
            
            for i in range(0, self.joystick.get_numaxes()):
                val = self.joystick.get_axis(i)
                if abs(val) < 0.2:
                    val = 0
                tmp = "self.cmd.axis" + str(i) + " = " + str(val)
                if val != 0:
                    exec(tmp)
                
            for i in range(0, self.joystick.get_numhats()):
                val = self.joystick.get_hat(i)
                tmp = "self.cmd.hat" + str(i) + " = " + str(val)
                exec(tmp)
                    
            for i in range(0, self.joystick.get_numbuttons()):
                if self.joystick.get_button(i) != 0:
                    tmp = "self.cmd.btn" + str(i) + " = 1"
                    exec(tmp)

            if self.cmd.btn1:
                #toggle IO
                air = not air
                self.robot.speedStop()

            if self.cmd.btn9:
                 self.robot.moveJ([0,-math.pi/2,math.pi/2,0,math.pi/2,math.pi],0.5,0.1)
                                      
            if self.cmd.btn0 and self.cmd.hat0 == (0, 1):
                print("value from the hat is,self.cmd.btn0",self.cmd.hat0,self.cmd.btn0)
                self.move_robo([0,0,0.1,0,0,0])
            elif self.cmd.btn0 and self.cmd.hat0 == (0, -1):
                print("value from the hat is,self.cmd.hat0",self.cmd.hat0,self.cmd.btn0)
                self.move_robo([0,0,-0.1,0,0,0])
            elif self.cmd.hat0 == (0, 1):
                print("value from the hat is",self.cmd.hat0)
                self.test("test")
                self.move_robo([0,-0.1,0,0,0,0])
            elif self.cmd.hat0 == (0, -1):
                print("value from the hat is",self.cmd.hat0)
                self.move_robo([0,0.1,0,0,0,0])
            elif self.cmd.hat0 == (1, 0):
                print("value from the hat is",self.cmd.hat0)
                self.move_robo([-0.1,0,0,0,0,0])
            elif self.cmd.hat0 == (-1, 0):
                print("value from the hat is",self.cmd.hat0)
                self.move_robo([0.1,0,0,0,0,0])
            elif self.cmd.axis0 > 0.5 or self.cmd.axis0 < -0.5:
                self.rotate_robo([0,0,0,self.cmd.axis0,0,0])
                print("movement in axis0")
            elif self.cmd.axis1 > 0.5 or self.cmd.axis1 < -0.5:
                self.rotate_robo([0,0,0,0,self.cmd.axis1,0])
            elif self.cmd.axis2 > 0.5 or self.cmd.axis2 < -0.5:
                self.rotate_robo([0,0,0,0,0,self.cmd.axis2])
            else:
                self.robot.speedStop()
                
                            
           # if self.cmd.axis0 > 0.2 or self.cmd.axis0 > -0.2 :
           #     self.rotate_robo([0,0,0,self.cmd.axis0,0,0])
           # elif self.cmd.axis1 > 0.2 or self.cmd.axis1 > -0.2 :
           #     self.rotate_robo([0,0,0,0,self.cmd.axis1,0])
           # else:
           #     self.robot.speedStop()
            #elif self.cmd.axis2 != 0:
                #self.rotate_robo([0,0,0,0,0,self.cmd.axis2])
            
            

    def close(self):
        if self.joystick:
            self.joystick.quit()

In [ ]:
if __name__ == "__main__":
    robot = rtde_control.RTDEControlInterface("127.0.0.1")
    robo_io = rtde_io.RTDEIOInterface("127.0.0.1")
    rtde_r = rtde_receive.RTDEReceiveInterface("127.0.0.1")
    robo_io = 0
    r = robot
    

    service = Service(robot,robo_io, rtde_r, linear_velocity=0.1, rotational_velocity=0.1, acceleration=0.1)
    service.init_joystick()
    try:
        service.loop() 
    finally: 
        robot.close()
        service.close()

In [ ]:

rtde_c.moveJ([0,-math.pi/2,math.pi/2,0,math.pi/2,math.pi], 1, 1)